# Importing the Libraries

In [ ]:
import os
import sys
import math
import numpy as np
import re
import random
import json
import en_core_web_sm
import pandas as pd
import torch
import spacy
!pip install sentencepiece 
!pip install transformers
import sentencepiece
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
)

     |████████████████████████████████| 1.2 MB 7.0 MB/s 
     |████████████████████████████████| 2.6 MB 8.0 MB/s 
     |████████████████████████████████| 636 kB 48.6 MB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 
     |████████████████████████████████| 3.3 MB 52.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Question Generator class definition using the T5 pretrained model

In [ ]:
class QG:
    def __init__(self):
        self.SeqLength= 512
        self.qa_evaluator = QAEvaluator()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.AnswerToken = "<answer>"
        self.ContentToken = "<context>"
        QG_Pretrained = "QG_Pretrained_t5_model_trained.pth"
        self.qg_tokenizer = AutoTokenizer.from_pretrained(QG_Pretrained, use_fast = False)
        self.qg_model = AutoModelForSeq2SeqLM.from_pretrained(QG_Pretrained)
        self.qg_model.to(self.device)

    def generator(self, article, useEvaluator = True, numQuestions = None):
        print("Creating the questions using T-5 \n")
        qgInputs, qgAnswers = self.generateInputs(article)
        generatedQuestions = self.generateQuesFromInputs(qgInputs)
        if useEvaluator:
            print("Evaluating the QA pairs using BERT \n")
            encoded_qa_pairs = self.qa_evaluator.encodeQApairs(generatedQuestions, qgAnswers)
            scores = self.qa_evaluator.getScores(encoded_qa_pairs)
            if numQuestions:
                qaList = self.getRankedPairs(generatedQuestions, qgAnswers, scores, numQuestions)
            else:
                qaList = self.getRankedPairs(generatedQuestions, qgAnswers, scores)
        else:
            print("Skipping the evaluation step \n")
            qaList = self.getAllPairs(generatedQuestions, qgAnswers)
        return qaList

    def generateInputs(self, text):
        inputs = []
        answers = []
        segments = self.splitToSegments(text)
        for segment in segments:
            sentences = self.splitText(segment)
            prepped_inputs, prepped_answers = self.prepareInputs(sentences, segment)
            inputs.extend(prepped_inputs)
            answers.extend(prepped_answers)
        return inputs, answers

    def splitToSegments(self, text):
        MaxTokens = 490
        paragraphs = text.split("\n")
        tokenizedParagraphs = [self.qg_tokenizer(p)["input_ids"] for p in paragraphs if len(p) > 0]
        segments = []
        while len(tokenizedParagraphs) > 0:
            segment = []
            while len(segment) < MaxTokens and len(tokenizedParagraphs) > 0:
                paragraph = tokenizedParagraphs.pop(0)
                segment.extend(paragraph)
            segments.append(segment)
        return [self.qg_tokenizer.decode(s) for s in segments]

    def generateQuesFromInputs(self, qgInputs):
        generatedQuestions = []
        for qg_input in qgInputs:
            question = self.generateQuestion(qg_input)
            generatedQuestions.append(question)
        return generatedQuestions

    def splitText(self, text):
        MaxSentenceLen = 128
        sentences = re.findall(".*?[.!\?]", text)
        cut_sentences = []
        for sentence in sentences:
            if len(sentence) > MaxSentenceLen:
                cut_sentences.extend(re.split("[,;:)]", sentence))
        cut_sentences = [s for s in sentences if len(s.split(" ")) > 5]
        sentences = sentences + cut_sentences
        return list(set([s.strip(" ") for s in sentences]))
      
    def getRankedPairs(self, generatedQuestions, qgAnswers, scores, numQuestions=10):
        if numQuestions > len(scores):
            numQuestions = len(scores)
            print("\nWas only able to generate {} questions. For more questions, please input a longer text.".format(numQuestions))
        qaList = []
        for i in range(numQuestions):
            index = scores[i]
            qa = self.makeDict(generatedQuestions[index].split("?")[0] + "?", qgAnswers[index])
            qaList.append(qa)
        return qaList

    def prepareInputs(self, sentences, text):
        inputs = []
        answers = []
        for sentence in sentences:
            qg_input = "{} {} {} {}".format(self.AnswerToken, sentence, self.ContentToken, text)
            inputs.append(qg_input)
            answers.append(sentence)
        return inputs, answers

    def generateQuestion(self, qg_input):
        self.qg_model.eval()
        encoded_input = self.encodeQG(qg_input)
        with torch.no_grad():
            output = self.qg_model.generator(input_ids=encoded_input["input_ids"])
        question = self.qg_tokenizer.decode(output[0], skip_special_tokens=True)
        return question

    def encodeQG(self, qg_input):
        return self.qg_tokenizer(
            qg_input,
            padding='max_length',
            max_length=self.SeqLength,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

    def makeDict(self, question, answer):
        qa = {}
        qa["question"] = question
        qa["answer"] = answer
        return qa

    def getAllPairs(self, generatedQuestions, qgAnswers):
        qaList = []
        for i in range(len(generatedQuestions)):
            qa = self.makeDict(generatedQuestions[i].split("?")[0] + "?", qgAnswers[i])
            qaList.append(qa)
        return qaList


# QA Evaluator class definition using the BERT pretrained model

In [ ]:
class QAEvaluator:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        evalPretrained = "qa_eval_model_trained_bert_large.pt"
        self.SeqLength = 512
        self.evalTokenizer = AutoTokenizer.from_pretrained(evalPretrained)
        self.qae_model = AutoModelForSequenceClassification.from_pretrained(evalPretrained)
        self.qae_model.to(self.device)

    def encodeQApairs(self, questions, answers):
        encoded_pairs = []
        for i in range(len(questions)):
            encoded_qa = self.encodeQA(questions[i], answers[i])
            encoded_pairs.append(encoded_qa.to(self.device))
        return encoded_pairs

    def getScores(self, encoded_qa_pairs):
        scores = {}
        self.qae_model.eval()
        with torch.no_grad():
            for i in range(len(encoded_qa_pairs)):
                scores[i] = self.evaluateQA(encoded_qa_pairs[i])
        return [k for k, v in sorted(scores.items(), key = lambda item: item[1], reverse = True)]

    def encodeQA(self, question, answer):
        if type(answer) is list:
            for a in answer:
                if a["correct"]:
                    correct_answer = a["answer"]
        else:
            correct_answer = answer
        return self.evalTokenizer(
            text = question,
            text_pair = correct_answer,
            padding = "max_length",
            max_length = self.SeqLength,
            truncation = True,
            return_tensors = "pt",
        )

    def evaluateQA(self, encoded_qa_pair):
        output = self.qae_model(**encoded_qa_pair)
        return output[0][0][1]


# Evaluation

## Making an object of QG class

In [ ]:
qg = QG()

## Testing the Question Generation on Sample Passages

In [ ]:
passage1 = '''
The COVID-19 (coronavirus) outbreak has significantly affected the global economy. First identified in Wuhan, China, the coronavirus has spread globally, with Ireland no exception. This article examines the impact of the coronavirus outbreak on the Irish economy.

The Irish government has reported 15,251 cases and 610 fatalities as of 19 April, significantly lower than over 103,000 confirmed cases across the United Kingdom. The daily rise in the number of cases has prompted a number of measures aimed at stemming the spread of the virus and easing the pressure on the Health Service Executive. In mid-March, less than a month after the first confirmed case in the country, the Irish government announced the closure of schools and non-essential businesses including pubs, bars and hotels. At the end of the month, measures became more stringent with all individuals told to stay at home unless for essential trips, such as food purchases and working for essential services. These containment measures implemented to limit the spread of the virus have led to severe economic shocks.

The halt to a large proportion of economic activity has led to a sharp increase in the national unemployment rate, to 5% in March according to the Central Statistics Office (CSO). The same body also stated that the COVID-19 adjusted unemployment rate was as high as 16.5%. Some of the most-exposed sectors, including retail, hospitality and construction, are expected to report up to 500,000 job losses before the end of April. Restaurants and takeaways have been particularly affected. However, a large proportion of workers in the financial services sector have been able to switch to remote working, placing employees in this sector at a lower risk of losing their jobs. This is crucial for the Irish economy, with professional, financial and support services accounting for approximately 42% of Irish GDP, according to the CSO. On 24 March, the Irish government announced the introduction of the Income Support Scheme, which provides a temporary subsidy of up to 70% of wages, a COVID-19 unemployment payment of €350 per week for employees and the self-employed to help relieve the strain on both businesses and consumers.
'''

qa_pairs_1 = qg.generator(passage1, numQuestions = 10)
qa_pairs_1

Generating questions...



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Evaluating QA pairs...



[{'answer': 'Some of the most-exposed sectors, including retail, hospitality and construction, are expected to report up to 500,000 job losses before the end of April.',
  'question': 'What sectors are expected to report the most job losses before the end of April?'},
 {'answer': '</s> The Irish government has reported 15,251 cases and 610 fatalities as of 19 April, significantly lower than over 103,000 confirmed cases across the United Kingdom.',
  'question': 'How many deaths have been reported in Ireland as of 19 April?'},
 {'answer': 'The COVID-19 (coronavirus) outbreak has significantly affected the global economy.',
  'question': 'What is the impact of the outbreak on the economy?'},
 {'answer': 'The daily rise in the number of cases has prompted a number of measures aimed at stemming the spread of the virus and easing the pressure on the Health Service Executive.',
  'question': 'What is the impact of the coronavirus on the Irish economy?'},
 {'answer': 'This is crucial for the 

In [ ]:
passage2 = '''
The Irish automotive sector has been profoundly affected by the lockdown measures. Prior to the crisis, sales were exhibiting a downward trend in the Motor Vehicle Dealers industry. The outbreak has decimated demand, with new vehicle registrations falling by 63.1% in March 2020 as just 6,150 vehicles were registered, down from 16,600 in the same month in the previous year, according to the Society of the Irish Motor Industry. As private consumption and consumer confidence decline sharply, expenditure on non-essential items such as new vehicles is anticipated to remain low in the short term.

Food supply in Ireland has been unaffected mainly because it is self-sufficient for much produce, and Irish supermarkets have reported a sharp uptick in demand as coronavirus case numbers have risen, leading to major players in the industry launching recruitment drives. Tesco and Musgrave have employed thousands of people to meet higher in-store and online demand. IBISWorld expects higher online retail sales to represent a fundamental change in the way customers shop for groceries, with a higher proportion of customers projected to spend more online after the measures are eased.

Major commercial flight operators Ryanair and Aer Lingus have grounded majority of their fleets as many countries have closed their borders, leading to a significant short-term decline in revenue for the Passenger Air Transport industry. Evidencing this, on Easter Monday just 900 passengers passed through Dublin Airport, compared with over 100,000 in the previous year.
'''

qa_pairs_2 = qg.generator(passage2, numQuestions = 10)
qa_pairs_2

Generating questions...



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Evaluating QA pairs...



[{'answer': '</s> Major commercial flight operators Ryanair and Aer Lingus have grounded majority of their fleets as many countries have closed their borders, leading to a significant short-term decline in revenue for the Passenger Air Transport industry.',
  'question': 'What is the impact of the closures on the Irish passenger transport industry?'},
 {'answer': '1% in March 2020 as just 6,150 vehicles were registered, down from 16,600 in the same month in the previous year, according to the Society of the Irish Motor Industry.',
  'question': 'How many vehicles were registered in March 2020?'},
 {'answer': 'Prior to the crisis, sales were exhibiting a downward trend in the Motor Vehicle Dealers industry.',
  'question': 'What is the impact of the crisis on the Irish automotive industry?'},
 {'answer': 'The outbreak has decimated demand, with new vehicle registrations falling by 63.',
  'question': 'How many people have been affected by the outbreak?'},
 {'answer': '</s> Food supply i

## Checking for the part of results by skipping the BERT evaluation of QA pairs generated from T5

In [ ]:
qa_pairs_1_without_BERT = qg.generator(passage1, useEvaluator = False, numQuestions = 10)
qa_pairs_1_without_BERT

Generating questions...



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Skipping evaluation step.



[{'answer': '</s> The Irish government has reported 15,251 cases and 610 fatalities as of 19 April, significantly lower than over 103,000 confirmed cases across the United Kingdom.',
  'question': 'How many deaths have been reported in Ireland as of 19 April?'},
 {'answer': '</s> The halt to a large proportion of economic activity has led to a sharp increase in the national unemployment rate, to 5% in March according to the Central Statistics Office (CSO).',
  'question': 'What is the impact of the COVID-19 on the Irish economy?'},
 {'answer': 'However, a large proportion of workers in the financial services sector have been able to switch to remote working, placing employees in this sector at a lower risk of losing their jobs.',
  'question': 'What is the impact of the outbreak on the Irish economy?'},
 {'answer': 'These containment measures implemented to limit the spread of the virus have led to severe economic shocks.',
  'question': 'What are the effects of the COVID-19 on the Iri

In [ ]:
qa_pairs_2_without_BERT = qg.generator(passage2, useEvaluator = False)
qa_pairs_2_without_BERT

Generating questions...



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Skipping evaluation step.



[{'answer': 'As private consumption and consumer confidence decline sharply, expenditure on non-essential items such as new vehicles is anticipated to remain low in the short term.',
  'question': 'What is the impact of the lockdown on the Irish automotive industry?'},
 {'answer': 'The outbreak has decimated demand, with new vehicle registrations falling by 63.',
  'question': 'How many people have been affected by the outbreak?'},
 {'answer': 'Prior to the crisis, sales were exhibiting a downward trend in the Motor Vehicle Dealers industry.',
  'question': 'What is the impact of the crisis on the Irish automotive industry?'},
 {'answer': 'Evidencing this, on Easter Monday just 900 passengers passed through Dublin Airport, compared with over 100,000 in the previous year.',
  'question': 'How many passengers passed through Dublin Airport on Easter Monday?'},
 {'answer': 'Tesco and Musgrave have employed thousands of people to meet higher in-store and online demand.',
  'question': 'How 

## Creating the QA pairs samples for Extrinsic Evaluation


In [ ]:
# Setting up the random sample qa pair size from single passage
random_sample_size = 5

# Creating the QA pairs list to store the 10 random pairs to be used
qa_pairs_1_list = []
qa_pairs_2_list = []
for i in range(10):
  qa_pairs_1_list.append(random.sample(qa_pairs_1, random_sample_size))
  qa_pairs_2_list.append(random.sample(qa_pairs_2, random_sample_size))

In [ ]:
evaluation_set_1 = pd.DataFrame(qa_pairs_1_list)
evaluation_set_2 = pd.DataFrame(qa_pairs_2_list)

In [ ]:
evaluation_set = pd.concat([evaluation_set_1, evaluation_set_2], axis = 1)

In [ ]:
evaluation_set

,0,1,2,3,4,0,1,2,3,4
0,{'question': 'What are the effects of the COVI...,{'question': 'What is the impact of the outbre...,{'question': 'How much of the GDP of Ireland i...,{'question': 'How much unemployment did the go...,{'question': 'What is the impact of the COVID-...,{'question': 'What is the impact of the lockdo...,{'question': 'Why is the food supply in Irelan...,{'question': 'How many vehicles were registere...,{'question': 'What is the impact of the closur...,{'question': 'How many passengers passed throu...
1,{'question': 'How much unemployment did the go...,{'question': 'What sectors have been particula...,{'question': 'What sectors are expected to rep...,{'question': 'What is the impact of the outbre...,{'question': 'How much of the GDP of Ireland i...,{'question': 'How many vehicles were registere...,{'question': 'What is the impact of the lockdo...,{'question': 'Why is the food supply in Irelan...,{'question': 'How many passengers passed throu...,{'question': 'What is the impact of the lockdo...
2,"{'question': 'Where did the outbreak occur?', ...",{'question': 'What sectors are expected to rep...,{'question': 'How much of the GDP of Ireland i...,{'question': 'How much unemployment did the go...,{'question': 'What are the effects of the COVI...,{'question': 'Why is the food supply in Irelan...,{'question': 'What is the impact of the crisis...,{'question': 'How many people have been employ...,{'question': 'How many passengers passed throu...,{'question': 'What is the impact of the lockdo...
3,{'question': 'What sectors are expected to rep...,"{'question': 'Where did the outbreak occur?', ...",{'question': 'How much unemployment did the go...,{'question': 'How many deaths have been report...,{'question': 'What sectors have been particula...,{'question': 'How many vehicles were registere...,{'question': 'What is the impact of the crisis...,{'question': 'How many people have been affect...,{'question': 'Why is the food supply in Irelan...,{'question': 'How many passengers passed throu...
4,{'question': 'What is the impact of the outbre...,{'question': 'How much unemployment did the go...,"{'question': 'Where did the outbreak occur?', ...",{'question': 'What is the impact of the corona...,{'question': 'How much of the GDP of Ireland i...,{'question': 'Why is the food supply in Irelan...,{'question': 'What is the impact of the crisis...,{'question': 'How many passengers passed throu...,{'question': 'What is the impact of the closur...,{'question': 'What is the impact of the lockdo...
5,{'question': 'How many deaths have been report...,{'question': 'What sectors have been particula...,{'question': 'What is the impact of the corona...,{'question': 'What is the impact of the outbre...,"{'question': 'Where did the outbreak occur?', ...",{'question': 'What is the impact of the lockdo...,{'question': 'What is the impact of the lockdo...,{'question': 'How many people have been employ...,{'question': 'Why is the food supply in Irelan...,{'question': 'What is the impact of the closur...
6,{'question': 'How much of the GDP of Ireland i...,{'question': 'How many deaths have been report...,{'question': 'What is the impact of the COVID-...,"{'question': 'Where did the outbreak occur?', ...",{'question': 'What sectors are expected to rep...,{'question': 'Why is the food supply in Irelan...,{'question': 'How many passengers passed throu...,{'question': 'What is the impact of the crisis...,{'question': 'What is the impact of the lockdo...,{'question': 'How many people have been employ...
7,{'question': 'What is the impact of the COVID-...,{'question': 'What are the effects of the COVI...,{'question': 'What is the impact of the outbre...,{'question': 'What sectors are expected to rep...,{'question': 'How much of the GDP of Ireland i...,{'question': 'What is the impact of the crisis...,{'question': 'How many people have been employ...,{'question': 'What is the impact of the closur...,{'question': 'How many people have been affect...,{'q

In [ ]:
DIR = os.path.expanduser('/content/gdrive/MyDrive/QG_dataset/')
evaluation_set.to_csv(os.path.join(DIR, 'Human_Evaluation_QA_set.csv')) 

## Entire set of QA pairs generated from both the passages

In [ ]:
qa_pairs = qa_pairs_1 + qa_pairs_2
qa_pairs

[[{'answer': 'Some of the most-exposed sectors, including retail, hospitality and construction, are expected to report up to 500,000 job losses before the end of April.',
   'question': 'What sectors are expected to see the most job losses?'},
  {'answer': '</s> The Irish government has reported 15,251 cases and 610 fatalities as of 19 April, significantly lower than over 103,000 confirmed cases across the United Kingdom.',
   'question': 'How many deaths have been reported in Ireland as of 19 April?'},
  {'answer': 'The COVID-19 (coronavirus) outbreak has significantly affected the global economy.',
   'question': 'What is the impact of the outbreak on the economy?'},
  {'answer': 'The daily rise in the number of cases has prompted a number of measures aimed at stemming the spread of the virus and easing the pressure on the Health Service Executive.',
   'question': 'What is the impact of the coronavirus on the Irish economy?'},
  {'answer': 'This is crucial for the Irish economy, wit

In [ ]:
A = []
Q = []
QA = pd.DataFrame()
for i in range(len(qa_pairs)):
  Q.append(qa_pairs[i]['question'])
  A.append(qa_pairs[i]['answer'])
QA['Question'] = Q
QA['Answer'] = A

In [ ]:
QA

,Question,Answer
0,What sectors are expected to report the most j...,"Some of the most-exposed sectors, including re..."
1,How many deaths have been reported in Ireland ...,"</s> The Irish government has reported 15,251 ..."
2,What is the impact of the outbreak on the econ...,The COVID-19 (coronavirus) outbreak has signif...
3,What is the impact of the coronavirus on the I...,The daily rise in the number of cases has prom...
4,How much of the GDP of Ireland is affected by ...,"This is crucial for the Irish economy, with pr..."
5,What sectors have been particularly affected b...,Restaurants and takeaways have been particular...
6,What is the impact of the COVID-19 on the Iris...,</s> The halt to a large proportion of economi...
7,What are the effects of the COVID-19 on the Ir...,These containment measures implemented to limi...
8,Where did the outbreak occur?,"First identified in Wuhan, China, the coronavi..."
9,How much unemployment did the government expec...,5%.


## Intrinsic/Automatic Evaluation Metrics

### BLEU score

In [ ]:
# Importing the packages required for the BLEU score
import nltk
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

In [ ]:
# Applying the bleu score metric to the QA pairs list 
Bleu_Score = []
for i in range(len(QA['Question'])):
  score = bleu([QA['Question'][i]], QA['Answer'][i], smoothing_function=smoothie)
  Bleu_Score.append(round(score*100,2))

In [ ]:
# Bleu Score Matrix for the QA pairs generated
Bleu_Score

[46.11,
 19.95,
 28.62,
 17.1,
 15.33,
 59.54,
 13.36,
 20.27,
 6.95,
 0,
 19.74,
 23.07,
 29.26,
 26.2,
 10.72,
 43.38,
 29.93,
 11.4,
 61.81,
 9.02]

### ROGUE score

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

In [ ]:
rouge.getScores(QA['Question'], QA['Answer'], avg=True)

{'rouge-1': {'f': 0.21703264418708773,
  'p': 0.3158910533910534,
  'r': 0.17882853910795088},
 'rouge-2': {'f': 0.09470521661014109,
  'p': 0.1329202741702742,
  'r': 0.08313771259740485},
 'rouge-l': {'f': 0.18439040634078263,
  'p': 0.26489357864357865,
  'r': 0.15297872225813408}}

In [ ]:
# Rogue results for passage 1
rouge.getScores(QA['Question'][:10], QA['Answer'][:10], avg=True)

{'rouge-1': {'f': 0.2271449077862763,
  'p': 0.301984126984127,
  'r': 0.1957161172161172},
 'rouge-2': {'f': 0.11516021292091938,
  'p': 0.14282467532467533,
  'r': 0.1059291604197901},
 'rouge-l': {'f': 0.2206932948830505,
  'p': 0.291984126984127,
  'r': 0.19095421245421246}}

In [ ]:
# Rogue results for passage 2
rouge.getScores(QA['Question'][10:], QA['Answer'][10:], avg=True)

{'rouge-1': {'f': 0.2069203805878992,
  'p': 0.3297979797979798,
  'r': 0.16194096099978453},
 'rouge-2': {'f': 0.07425022029936282,
  'p': 0.12301587301587302,
  'r': 0.06034626477501959},
 'rouge-l': {'f': 0.1480875177985147,
  'p': 0.2378030303030303,
  'r': 0.11500323206205557}}

### METEOR score

In [ ]:
!pip install -U nltk
import nltk.translate.meteor_score as meteor
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Applying the meteor score metric to the QA pairs list 
Meteor_Score = []
for i in range(len(QA['Question'])):
  score = meteor.single_meteor_score(QA['Answer'][i], QA['Question'][i])
  Meteor_Score.append(round(score*100,2))

In [ ]:
Meteor_Score

[45.38,
 15.24,
 15.0,
 12.83,
 8.77,
 40.9,
 6.69,
 9.15,
 3.82,
 0.0,
 5.93,
 11.35,
 6.41,
 8.55,
 11.02,
 30.99,
 10.27,
 6.33,
 30.74,
 6.31]

### Jaccard Similarity

In [ ]:
# Importing the packages required for the Jaccard Similarity
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams

In [ ]:
# Applying the jaccard similarity metric to the QA pairs list 
Jaccard_Similarity = []
for i in range(len(QA['Question'])):
  JS = 1-jaccard_distance(set(ngrams(QA['Question'][i], 1)), set(ngrams(QA['Answer'][i], 1)))
  Jaccard_Similarity.append(round(JS*100,2))

In [ ]:
# Jaccard Similarity Matrix for the QA pairs generated
Jaccard_Similarity

[64.52,
 55.0,
 47.22,
 56.67,
 57.14,
 76.0,
 40.48,
 40.62,
 43.33,
 0.0,
 51.35,
 60.0,
 43.75,
 63.33,
 50.0,
 69.7,
 51.72,
 68.97,
 77.78,
 66.67]

### Cosine Similarity

In [ ]:
!pip install textdistance
import textdistance

In [ ]:
# Applying the cosine similarity metric to the QA pairs list 
Cosine_Similarity = []
for i in range(len(QA['Question'])):
  CS = 1-textdistance.Cosine(qval=1).distance(QA['Question'][i], QA['Answer'][i])
  Cosine_Similarity.append(round(CS*100,2))

In [ ]:
# Cosine Similarity Matrix for the QA pairs generated
Cosine_Similarity

[70.27,
 57.47,
 67.15,
 54.92,
 52.99,
 79.02,
 46.3,
 55.85,
 50.91,
 0.0,
 53.48,
 51.38,
 68.39,
 67.32,
 35.79,
 68.86,
 64.9,
 60.06,
 82.88,
 48.87]

## Creating the evaluation matrix by joining the individual matrix 

In [ ]:
Evaluations = pd.DataFrame()
Evaluations['Question'] = QA['Question']
Evaluations['Answer'] = QA['Answer']
Evaluations['Bleu_Score'] = Bleu_Score
Evaluations['Meteor_Score'] = Meteor_Score
Evaluations['Jaccard_Similarity'] = Jaccard_Similarity
Evaluations['Cosine_Similarity'] = Cosine_Similarity

In [ ]:
Evaluations

,Question,Answer,Bleu_Score,Meteor_Score,Jaccard_Similarity,Cosine_Similarity
0,What sectors are expected to report the most j...,"Some of the most-exposed sectors, including re...",46.11,45.38,64.52,70.27
1,How many deaths have been reported in Ireland ...,"</s> The Irish government has reported 15,251 ...",19.95,15.24,55.00,57.47
2,What is the impact of the outbreak on the econ...,The COVID-19 (coronavirus) outbreak has signif...,28.62,15.00,47.22,67.15
3,What is the impact of the coronavirus on the I...,The daily rise in the number of cases has prom...,17.10,12.83,56.67,54.92
4,How much of the GDP of Ireland is affected by ...,"This is crucial for the Irish economy, with pr...",15.33,8.77,57.14,52.99
5,What sectors have been particularly affected b...,Restaurants and takeaways have been particular...,59.54,40.90,76.00,79.02
6,What is the impact of the COVID-19 on the Iris...,</s> The halt to a large proportion of economi...,13.36,6.69,40.48,46.30
7,What are the effects of the COVID-19 on the Ir...,These containment measures implemented to limi...,20.27,9.15,40.62,55.85
8,Where did the outbreak occur?,"First identified in Wuhan, China, the coronavi...",6.95,3.82,43.33,50.91
9,How much unemployment did the government expec...,5%.,0.00,0.00,0.00,0.00


### Mount colab notebook to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
DIR = os.path.expanduser('/content/gdrive/MyDrive/QG_dataset/')
Evaluations.to_csv(os.path.join(DIR, 'Evaluation_Matrix.csv')) 